<a href="https://colab.research.google.com/github/kkrueger/Redis-Workshops/blob/main/05-LangChain_Redis/05.2_Dolly_v2_LangChain_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Question Answering with LangChain, Dolly LLM and Redis

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook uses `databricks/dolly-v2-3b` the LLM from HuggingFace, Redis with Vector Similarity Search, and LangChain to answer questions about the information contained in a document.

### Install Dependencies

WARNING!!! We have to downgrade the version of one of the libraries here and that requires restart of the notebook runtime. At the end of the next cell you'll be prompted to restart the runtime. Restart and continue with the execution.

In [ ]:
!pip install -q redis "langchain==0.0.227" tiktoken sentence_transformers transformers accelerate


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)



### Install Redis Stack

Redis will be used as Vector Similarity Search engine for LangChain. Instead of using in-notebook Redis Stack you can provision your own free instance of Redis in the cloud. Get your own Free Redis Cloud instance at https://redis.com/try-free/

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [ ]:
import redis
import os


REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD=""

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

In [ ]:
import os
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt

### Load text and split it into manageable chunks

Without this step any large body of text would exceed the limit of tokens you can feed to the LLM

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

### Initialize embeddings engine

Here we are using https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 which is a compact embedding engine that can be run locally. For large number of embeddings/documents you might consider GPU - enabled runtime or calling the hosted Embeddings API.

In [ ]:
def get_embeddings():
  from langchain.embeddings import HuggingFaceEmbeddings
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  return embeddings

embeddings = get_embeddings()

### Initialize LLM

In this notebook we are using VertexAI from Google Cloud.

In [ ]:
llm = HuggingFacePipeline(pipeline=generate_text)

### Create vector store from the documents using Redis as Vector Database

In [ ]:
def get_vectorstore() -> Redis:
    """Create the Redis vectorstore."""

    try:
        vectorstore = Redis.from_existing_index(
            embedding=embeddings,
            index_name=INDEX_NAME,
            redis_url=REDIS_URL
        )
        return vectorstore
    except:
        pass

    # Load Redis with documents
    vectorstore = Redis.from_documents(
        documents=texts,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectorstore


redis = get_vectorstore()

## Specify the prompt template

PromptTemplate defines the exact text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [ ]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the LangChain brings all the components together in a form of a simple QnA chain

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()}
    )

### Debugging Redis

The code block below is example of how you can interact with the Redis Database

In [ ]:
#!redis-cli $REDIS_CONN keys "*"
#!redis-cli $REDIS_CONN HGETALL "doc:qna:idx:063955c855a7436fbf9829821332ed2a"

###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
#!redis-cli $REDIS_CONN flushdb


## Finally - let's ask questions!

Examples:
- What did the president say about Kentaji Brown Jackson
- Did he mention Stephen Breyer?
- What was his stance on Ukraine

In [ ]:


query = "What did the president say about Kentaji Brown Jackson"
qa.run(query)